Here is the code to analyse how noise depends on the target size


In [ ]:
##Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os

In [ ]:
folder_path = '/home/myedut/Downloads/sinewave_old/'

# Initialize the list
chenyao_data_raw_3 = []

# Use glob to find all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Read each CSV file and append to the list
for file in csv_files:
    chenyao_data_raw_3.append(pd.read_csv(file))

folder_path = '/home/myedut/Downloads/sinewave_new/'
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))



for file in csv_files:
    chenyao_data_raw_3.append(pd.read_csv(file))

In [ ]:
###Pick ranges that contain flying with sinewave-modulated acceleration mu_x
ranges=[(324,710),(100,420),(100,620),(100,710),(100,600)]

ranges_2=[(50,250),(50,400),(50,250),(50,300),(60,360),(60,360),(60,310),(60,310)]

ranges_3=ranges+ranges_2

ranges_2=ranges_3

In [ ]:
#Downsample the data
chenyao_data_resampled = []

# Ensure there is no variable named 'list' that shadows the built-in list type
# If you have such a variable, rename it to something else

for i in range(len(chenyao_data_raw_3)):
    print(i)
    chenyao_data_raw_3[i]['time'] = pd.to_timedelta(chenyao_data_raw_3[i]['time'], unit='s')
    chenyao_data_raw_3[i].set_index('time', inplace=True)

    chenyao_data_resampled.append(chenyao_data_raw_3[i].resample('65ms').mean())

In [ ]:
box_location_1=np.array([4,-.5,-0.7])

box_location_2=np.array([4.4,2.1,-1])
###Calculate the absolute distance to the landing target
for i in range(len(chenyao_data_resampled)):
    if i <5:
        box_location=box_location_1   
        true_x=(chenyao_data_resampled[i]['pos_x'])-box_location[0]
        true_y=chenyao_data_resampled[i]['pos_y']-box_location[1]
        true_z=chenyao_data_resampled[i]['pos_z']-box_location[2]
        chenyao_data_resampled[i]['abs_distance']=np.sqrt(true_x**2+true_y**2+true_z**2)
    else:
        box_location=box_location_2
        true_x=(chenyao_data_resampled[i]['pos_x'])-box_location[0]
        true_y=chenyao_data_resampled[i]['pos_y']-box_location[1]
        true_z=chenyao_data_resampled[i]['pos_z']-box_location[2]
        chenyao_data_resampled[i]['abs_distance']=np.sqrt(true_x**2+true_y**2+true_z**2)

#Calculate ground truth divergence based on your distance to the target and velocity. We first calculate velocity towards the box and then convert it to divertgence
for i in range(len(chenyao_data_resampled)):
    if i <5:
        box_location=box_location_1
        approach=((chenyao_data_resampled[i]['pos_x']-box_location[0])*chenyao_data_resampled[i]['vel_x'] + (chenyao_data_resampled[i]['pos_y']-box_location[1])*chenyao_data_resampled[i]['vel_y']+(chenyao_data_resampled[i]['pos_z']-box_location[2])*chenyao_data_resampled[i]['vel_z'])/chenyao_data_resampled[i]['abs_distance']
        chenyao_data_resampled[i]['real_divergence']=-1*approach/chenyao_data_resampled[i]['abs_distance']
    else:
        box_location=box_location_2
        approach=((chenyao_data_resampled[i]['pos_x']-box_location[0])*chenyao_data_resampled[i]['vel_x'] + (chenyao_data_resampled[i]['pos_y']-box_location[1])*chenyao_data_resampled[i]['vel_y']+(chenyao_data_resampled[i]['pos_z']-box_location[2])*chenyao_data_resampled[i]['vel_z'])/chenyao_data_resampled[i]['abs_distance']
        chenyao_data_resampled[i]['real_divergence']=-1*approach/chenyao_data_resampled[i]['abs_distance']
  

In [ ]:
###Calculate errpors
for i in range(len(chenyao_data_resampled)):
    chenyao_data_resampled[i]['absolute_error']=np.abs(chenyao_data_resampled[i]['raw_divergence'].shift(-4)-chenyao_data_resampled[i]['real_divergence'])
    chenyao_data_resampled[i]['square_error']=(chenyao_data_resampled[i]['raw_divergence'].shift(-4)-chenyao_data_resampled[i]['real_divergence'])**2
    chenyao_data_resampled[i]['relative_error']=chenyao_data_resampled[i]['absolute_error']/np.abs(chenyao_data_resampled[i]['real_divergence'])

In [ ]:


# Initialize lists to store pooled data
pooled_color_count = []
pooled_absolute_error = []
pooled_square_error = []
pooled_relative_error = []



# Loop through each DataFrame in chenyao_data_resampled
for i in range(len(chenyao_data_resampled)):
    # Extract the relevant data within the specified ranges
    color_count_data = chenyao_data_resampled[i]['color_count'].iloc[ranges_2[i][0]:ranges_2[i][1]].values
    absolute_error_data = chenyao_data_resampled[i]['absolute_error'].iloc[ranges_2[i][0]:ranges_2[i][1]].values
    square_error_data = chenyao_data_resampled[i]['square_error'].iloc[ranges_2[i][0]:ranges_2[i][1]].values
    relative_error_data = chenyao_data_resampled[i]['relative_error'].iloc[ranges_2[i][0]:ranges_2[i][1]].values
    
    # Append the data to the pooled lists
    pooled_color_count.extend(color_count_data)
    pooled_absolute_error.extend(absolute_error_data)
    pooled_square_error.extend(square_error_data)
    pooled_relative_error.extend(relative_error_data)

# Convert pooled lists to numpy arrays for plotting and fitting
pooled_color_count = np.array(pooled_color_count)
pooled_absolute_error = np.array(pooled_absolute_error)
pooled_square_error = np.array(pooled_square_error)
pooled_relative_error = np.array(pooled_relative_error)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming pooled_color_count, pooled_relative_error, pooled_absolute_error, and bins are already defined
bins = np.logspace(np.log10(pooled_color_count.min()), np.log10(pooled_color_count.max()), 15)
counts, bin_edges = np.histogram(pooled_color_count, bins=bins)
mid_bin_points = np.sqrt(bins[:-1] * bins[1:])

def calculate_statistics(pooled_color_count, pooled_error, bins):
    mean_noise = []
    sem_noise = []
    std_noise = []

    for i in range(len(bins) - 1):
        bin_indices = (pooled_color_count >= bins[i]) & (pooled_color_count < bins[i+1])
        bin_errors = pooled_error[bin_indices]
        mean_noise.append(np.mean(bin_errors))
        std_noise.append(np.std(bin_errors))
        sem_noise.append(std_noise[-1] / np.sqrt(len(bin_errors)))

    return mean_noise, sem_noise, std_noise

# Calculate statistics for relative error
mean_relative, sem_relative, std_relative = calculate_statistics(pooled_color_count, pooled_relative_error, bins)

# Calculate statistics for absolute error
mean_absolute, sem_absolute, std_absolute = calculate_statistics(pooled_color_count, pooled_absolute_error, bins)

# Plotting relative and absolute errors one under another
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 16))

# Plotting absolute error with linear Y scale, STD bars, and no legend
ax1.errorbar(mid_bin_points, mean_absolute, yerr=std_absolute, fmt='o', color='coral', linewidth=3, markersize=20, markerfacecolor='coral', capsize=12, capthick=3)
ax1.plot(mid_bin_points, mean_absolute, color='coral', linewidth=3)  # Add dotted line to the graph
ax1.set_xscale('log')
ax1.set_xlabel('Image Size (Number of pixels)', fontsize=22)
ax1.set_ylabel('Absolute Error', fontsize=22)
ax1.set_title('A. Absolute Error in Divergence Signal', fontsize=26)

# Remove top and right spines
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.tick_params(axis='both', which='major', labelsize=22)

# Plotting relative error with both axes on a logarithmic scale, SEM bars, and no legend
ax2.errorbar(mid_bin_points, mean_relative, yerr=sem_relative, fmt='o', color='coral', linewidth=3, markersize=20, markerfacecolor='coral', capsize=12, capthick=3)
ax2.plot(mid_bin_points, mean_relative, color='coral', linewidth=3)  # Add dotted line to the graph
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel('Image Size (Number of pixels)', fontsize=22)
ax2.set_ylabel('Relative Error', fontsize=22)
ax2.set_title('B. Relative Error in Divergence Signal', fontsize=26)

# Remove top and right spines
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.tick_params(axis='both', which='major', labelsize=22)

plt.tight_layout(pad=5.0)  # Increase space between plots
#plt.savefig('/home/myedut/Documents/Servoing_paper/Relative_and_Absolute_Noise_from_distance_figure_orange_vertical.png', dpi=300)
plt.show()